# Comparison of weighted Kemeny with different training methods

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
root_path = '../../'
sys.path.append(os.path.join(root_path, 'code', 'core'))


from ranking_utils import *
from mallows import *
from ws_ranking import *
import numpy as np

from synth_exp_utils import *
import pickle

import matplotlib
import matplotlib.pyplot as plt
plt.rc('axes', linewidth=2.5)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# Generate synthetic theta

In [2]:

d=10
n = 250
max_m = 18

np.random.seed(0)

theta_star = np.zeros(18)

theta_star[:10] = np.random.uniform(0.1,0.2,10)
theta_star[10:] = np.random.uniform(2,5,8)

lst_m = np.arange(2,19,2)
print(lst_m)
lst_out = []
np.sort(theta_star)
print(theta_star)
np.random.shuffle(theta_star)

[ 2  4  6  8 10 12 14 16 18]
[0.15488135 0.17151894 0.16027634 0.15448832 0.14236548 0.16458941
 0.14375872 0.1891773  0.19636628 0.13834415 4.37517511 3.58668476
 3.70413368 4.77678991 2.21310817 2.2613879  2.06065519 4.49785954]


# Run experiments

In [11]:

Y = generate_true_rankings(d,n)
L = sample_mallows_LFs(Y,max_m,theta_star)
lfs = {}
lst_D = get_pair_wise_dists(L)

#theta_hat = estimate_theta(L)

## Lines from estimate_theta
d = len(L[0][0])
m = len(L[0])
r_utils = RankingUtils(d)
wsr = WeakSupRanking(r_utils)
conf = {"train_method":"triplet_opt"}

# wsr.train(conf,L,m)
## Lines from wsr.train
n = len(L)  # the number of examples
if conf['train_method'] == 'triplet' or conf['train_method'] == 'triplet_opt':
    # TODO: handle the case when num LF  is not multiple of 3
    expected_dists = np.zeros(m)
    i = 0
    order, D = wsr.order_LFs_on_correlation(L,m)

    # solve 3 variable systems iteratively
    while i < m-2:
        l1, l2, l3 = order[i], order[i+1], order[i+2]
        #ac3 = self.solve_triplet(L, l1, l2, l3, D)
        ## Lines from ac3
        mu_12 = D[l1][l2]
        mu_23 = D[l2][l3]
        mu_31 = D[l3][l1]

        #ac3 = solve_3_var_system_sum(mu_12, mu_23, mu_31)
        ## Lines from solve_3_var_system_sum
        t = (mu_12 + mu_23 + mu_31) / 2
        ac3 = [t-mu_23,t-mu_31,t-mu_12]
        
        expected_dists[l1] = ac3[0]
        expected_dists[l2] = ac3[1]
        expected_dists[l3] = ac3[2]

        i += 3

    if conf['train_method'] == 'triplet':
        self.thetas = 1.0 / np.array(expected_dists)

    elif conf['train_method'] == 'triplet_opt':
        mlw = Mallows(r_utils, 1.0)
        logger.info("expected_dists {}".format(expected_dists))
        wsr.thetas = np.array([mlw.estimate_theta(d, expected_dists[i])
                                for i in range(len(expected_dists))])
        wsr.thetas = wsr.thetas.clip(1e-1, 100)

theta_hat = wsr.thetas

lst_trial_out = []
for m in lst_m:
    lst_trial_out.append(run_kemeny_inferences(Y,L,theta_star,theta_hat,m, lst_D))
o = collect_outs(lst_trial_out)

In [12]:
o

defaultdict(list,
            {'lst_mean_kt_unwtd_kemeny': [0.022577777777777776,
              0.01511111111111111,
              0.007199999999999999,
              0.0040888888888888884,
              0.0017777777777777776,
              0.000977777777777778,
              0.0008888888888888888,
              0.0014222222222222223,
              0.00017777777777777779],
             'lst_mean_kt_wtd_kemeny_opt': [0.006933333333333334,
              0.006933333333333334,
              0.0022222222222222222,
              0.0014222222222222223,
              0.00017777777777777779,
              8.888888888888889e-05,
              8.888888888888889e-05,
              0.0,
              0.0],
             'lst_mean_kt_wtd_kemeny_hat': [0.006933333333333334,
              0.006933333333333334,
              0.0022222222222222222,
              0.001511111111111111,
              0.00017777777777777779,
              8.888888888888889e-05,
              0.0,
              0.0,
         